### Grid search using gensim and visualize LDA results

In [ ]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings("ignore",category=RuntimeWarning)



In [ ]:
import pandas as pd
data_total = pd.read_csv(r"PerceptionSpamCleanedPipeline.csv", encoding = 'latin-1', engine = 'python', error_bad_lines=False)
len(data_total)

In [ ]:
data_total.columns

In [ ]:
df = data_total[data_total["IsSpam"] == 0]
print(len(df))

In [ ]:
#df = data_total[:10000]

In [ ]:
from nltk.util import ngrams
from collections import Counter
df['Contents'] = df['Contents'].str.replace('[^a-zA-Z\s@#$]','')
df['Contents'] = df['Contents'].apply(lambda x: [x for x in str(x).lower().split()])
df['Contents'] = df['Contents'].apply(lambda x: ngrams(x,2))
df['Contents'] = df['Contents'].apply(lambda x: [' '.join(item) for item in list(x)])



In [ ]:
df.Contents.iloc[4]

In [ ]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(df.Contents.tolist())

# Create Corpus
texts = df.Contents.tolist()

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        print(num_topics)
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, random_state=1)
        model_list.append(lda_model)
        coherencemodel = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=20, limit=100, step=5)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
Models_Coherence = [model_list, coherence_values]

In [ ]:
pkl_filename = "TopicModelsTwitter20_100_5_Bigrams.pkl"  
with open(pkl_filename, 'w') as file:  
    pickle.dump(Models_Coherence, file)

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5,
                                           random_state=1)

In [ ]:
#print(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=df.Contents.tolist(), dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("mallet-2.0.8.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [ ]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = r'mallet-2.0.8\\bin\\mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=5, id2word=id2word)

In [ ]:
# Load from file
with open(pkl_filename, 'rb') as file:  
    pickle_model = pickle.load(file)